<center>
<img style="float: center;" src="images/CI_horizontal.png" width="400">
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

<center> Julia Lane, Clayton Hunter, Brian Kim, Benjamin Feder, Ekaterina Levitskaya, Tian Lou, Lisa Osorio-Copete. 
</center>

# Unsupervised Machine Learning

There are problems where there does not exist a target variable that we want to predict, but instead we want to understand any "natural" grouping or patterns in the data. Unsupervised machine learning methods can help tackle these problems. Clustering is the most common unsupervised machine learning technique, but you might also be aware of principal components analysis (PCA) or neural networks implementations such as self-organizing maps (SOM). This notebook will provide an introduction to unsupervised machine learning through a clustering example.

## Introduction to Clustering

Clustering is used to group data points together that are similar to each other. Optimally, a given clustering method will produce groupings with high intra-cluster (within) similarity and low inter-cluster (between) similarity. Clustering algorithms typically require a distance or similarity metric to generate clusters. They take a dataset and a distance metric (and sometimes additional parameters), and they generate clusters based on that distance metric. The most common distance metric used is Euclidean distance, but other commonly used metrics are Manhattan, Minkowski, Chebyshev, cosine, Hamming, Pearson, and Mahalanobis.

Most clustering algorithms also require the user to specify the number of clusters (or some other parameter that indirectly determines the number of clusters) in advance as a parameter. This is often difficult to do a priori and typically makes clustering an iterative and interactive task. Another aspect of clustering that makes it interactive is often the difficulty in automatically evaluating the quality of the clusters. While varioius analytical clustering metrics have been developed, the best clustering is task-dependent and thus must be evaluated by the user. There may be different clusterings that can be generated with the same data. You can imagine clustering similar news stories based on the topic content, based on the writing style or based on sentiment. The right set of clusters depends on the user and the task they have. Clustering is therefore typically used for exploring the data, generating clusters, exploring the clusters, and then rerunning the clustering method with different parameters or modifying the clusters (by splitting or merging the previous set of clusters). Interpreting a cluster can be nontrivial: you can look at the centroid of a cluster, look at frequency distributions of different features (and compare them to the prior distribution of each feature).

Here, we will focus on **K-Means clustering** (*k* defines the number of clusters), which is considered to be the most commonly used clustering method. The algorithm works as follows:
1. Select *k* (the number of clusters you want to generate).
2. Initialize by selecting k points as centroids of the *k* clusters. This is typically done by selecting k points uniformly at random.
3. Assign each point a cluster according to the nearest centroid.
4. Recalculate cluster centroids based on the assignment in **(3)** as the mean of all data points belonging to that cluster.
5. Repeat **(3)** and **(4)** until convergence.

The algorithm stops when the assignments do not change from one iteration to the next. The final set of clusters, however, depends on the starting points. If initialized differently, it is possible that different clusters are obtained. One common practical trick is to run *k*-means several times, each with different (random) starting points. The *k*-means algorithm is fast, simple, and easy to use, and is often a good first clustering algorithm to try and see if it fits your needs. When the data are of the form where the mean of the data points cannot be computed, a related method called *K-medoids* can be used.

### Learning Objectives

This notebook demonstrates using *k*-means clustering to better understand Indiana's labor market in 2017 Q3. We've already developed a handful of employer-level measures in the [Employer-Level Measures](07_Employer_Level_Measures.ipynb) notebook. We will try a few different values of *k* to see how we can best understand the labor market by looking for differentiation between each of the clusters.

## Import Packages and Set Up


The main R package that we will use for clustering is called `cluster`. We also import all our usual packages for database connection and data manipulation/visualization.

In [ ]:
#database interaction imports
library(DBI)
library(RPostgreSQL)

# for data manipulation/visualization
library(tidyverse)
library(ggplot2)

# clustering
library(cluster)

In [ ]:
# create an RPostgreSQL driver
drv <- dbDriver("PostgreSQL")

# connect to the database
con <- dbConnect(drv,dbname = "postgresql://stuffed.adrf.info/appliedda")

## 1. Read in the Data

We will read-in a table from the database called `employers_2017` which contains characteristics of Indiana's labor market in 2017.

In [ ]:
# Read-in from the database
qry <- "
select *
from ada_tdc_2020.employers_2017
"
emp <- dbGetQuery(con, qry)

In [ ]:
# View first rows
head(emp)

This table contains information for employers by quarter. Because some employers appear in one quarter but may not appear in another quarter, for consistency, we will subset our dataframe to include information only for one quarter: third quarter of 2017.

In [ ]:
# Subset a dataframe by rows with 3rd quarter
# if you want to use the tidyverse way

emp <- emp %>%
    filter(quarter == 3)

In [ ]:
# Subset a dataframe by rows with 3rd quarter

# this is the same as the cell above; either method should produce the same result

emp <- emp[which(emp$quarter==3), ]

In [ ]:
# Check that we only have 3rd quarter now
unique(emp$quarter)

### Clean the Data

We need to remove the `uiacct` variable from our DataFrame since these features do not provide any explanatory power for our k-means algorithm. Additionally, k-means algorithms only work properly with continuous features. This is because k-means calculates its distance measure using euclidean distance, which is the distance between each data point and the centroid of a cluster. It is hard to assign positions for categorical variables in the euclidean space. Thus, we also need to remove `naics_3_digit` from `emp`.

> There are more complicated clustering algorithms that do not use Euclidean distances and thus allow categorical variables in the model. If you are interested in them, you can take a look at: `kmodes` function and `gower.dist`.

In [ ]:
# Remove uiacct, naics_3_digit columns, and also quarter column, as we are only working with one quarter
emp_ml <- emp %>%
    select(-c(uiacct, naics_3_digit, quarter))

In [ ]:
head(emp_ml)

In [ ]:
# Check data type of all variables - make sure all of them are numeric
str(emp_ml)

**It is important that we scale the features** before we compute *k*-means clustering, especially because the metrics are on a variety of numerical scales. 

In [ ]:
# Get descriptions of each variable using "summary" function
summary(emp_ml)

We can see that we have variables on different numerical scales - we can scale them using `scale()` function on our dataframe `emp_ml`.

In [ ]:
# Scale the features
emp_ml <- scale(emp_ml)

# View first rows after scaling
head(emp_ml)

In [ ]:
# Check number of rows (where each row is a unique employer)
nrow(emp_ml)

Before running a clustering algorithm, we need to make sure that there are no missing values. Here we will use `na.omit()` function which removes all rows with any NA values. (If an employer has missing information in any of the columns, a row will be dropped).

> Note that you should **never remove data** if possible - in a real world setting you would likely want to fill any missing data with an imputation or baseline assumption. We will discuss missing data during the Inference session in Module 3 of the program.

In [ ]:
# We also need to remove all missing data points before running clustering
# na.omit will remove any rows with any NA values
emp_ml <- na.omit(emp_ml)

In [ ]:
# Check number of rows after dropping rows with any NA values
nrow(emp_ml)

## 2. Choose the Number of Clusters, *K*

Running a *k*-means model is simple: we just need to use `kmeans()` and choose the number of clusters (called `centers`). What number should we choose? Here, we have 11 features, so it is hard to visualize the data and decide the proper number by using our eyes. Let's start with a small number, such as 3, and see how the results look like.  

Because *k*-means clustering will generate different results (due to different starting points), we will set a seed so that the work in this notebook can be reproducible using the `set.seed()`. To get the same results, you must use the same seed before running the clustering algorithm every time. Luckily, if you set the same seed as your collaborators and are running the same *k*-means algorithm, you will see the same results, even if you are working in different environments, i.e. Jupyter notebooks.

### k = 3

In [ ]:
# Initialize the model and run on emp_ml
set.seed(1)
k3 <- kmeans(emp_ml, centers=3, nstart=20)

> `nstart` specifies a number of initial configurations and reports on the best one - an optimal number is usually somewhere between 20 and 50. (See more information in the Resources section - Professor Steorts, Duke University).

In [ ]:
str(k3)

`kmeans` function returns the following components, most useful for us:
- `cluster` - an integer indicating a cluster to which each point is allocated
- `centers` - a matrix of cluster centers
- `totss` - the total sum of squares
- `withinss` - vector of within-cluster sum of squares, one component per cluster.
- `tot.withinss` - total within-cluster sum of squares, i.e. `sum(withinss)`
- `betweenss` - the between-cluster sum of squares, i.e. `totss-tot.withinss`
- `size` - the number of points in each cluster

Let's check the size of each cluster:

In [ ]:
k3$size

We can see that most of the employers are concentrated in cluster 3. In the perfect world, we would want them to be distributed more evenly across clusters, but in some cases, it may make sense that they wouldn't. Most importantly, we are looking for high intra-cluster similarity and low inter-cluster similarity.

Are there major differences in the characteristics of employers in each cluster?

We can take a look at basic descriptives of the employers in these clusters by adding our clustering results to the original dataframe, `emp`, and call this dataframe `frame_3`.

In [ ]:
emp <- na.omit(emp)                     # remove missing values
frame_3 <- data.frame(emp, k3$cluster)  # add cluster number to the original dataframe
frame_3 <- subset(frame_3, select= -c(uiacct,naics_3_digit, quarter))  # remove columns with uiacct, naics_3_digit, quarter

frame_3 %>%
    group_by(k3.cluster) %>%
    summarize_all("mean")

## Evaluate clusters

One simple way to evaluate clustering results is to compare the summary stats between key variables of interest.

We can also visualize the differences between the clusters in more detail by finding mean and standard deviation for the following variables: `avg_earnings`, `bottom_25_pctile`, and `top_25_pctile`. We will first need to convert our dataframe into a long format:

In [ ]:
# Save results with mean to a dataframe
frame_3_mean <- frame_3 %>%
    group_by(k3.cluster) %>%
    select(c(avg_earnings, bottom_25_pctile, top_25_pctile)) %>%
    summarize_all(mean) %>%
    pivot_longer(-k3.cluster, names_to = "variable", values_to = "mean")

# Save results with standard deviation to a dataframe
frame_3_sd <- frame_3 %>%
    group_by(k3.cluster) %>%
    select(c(avg_earnings, bottom_25_pctile, top_25_pctile)) %>%
    summarize_all(sd) %>%
    pivot_longer(-k3.cluster, names_to = "variable", values_to = "sd") %>%
    select(-c(k3.cluster, variable))

# Bind two dataframes together
df <- cbind(frame_3_mean,frame_3_sd)

df

Now we can use this dataframe to visualize mean and standard deviation in our 3 clusters by 3 variables: `avg_earnings`, `bottom_25_pctile`, and `top_25_pctile`:

In [ ]:
ggplot(df, aes(x=k3.cluster, y=mean, fill=k3.cluster)) +
    geom_bar(stat="identity", position = position_dodge()) +   # plot bars for the mean values
    geom_errorbar(aes(ymax= mean + sd, ymin = mean),            # add standard deviation bars
                  width=.2,
                  position = position_dodge(.9)) +
    facet_grid(. ~ variable) +                                  # plot by 3 variables of interest
    ggtitle("REDACTED") +  # add title
    xlab("Clusters") +                                          # add label for x-axis
    ylab("Mean") +                                              # add label for y-axis
    theme(text = element_text(size=16),                         # increase text font
          axis.text.x = element_text(size=18, face="bold"),     # increase text font on x-axis and make it bold
          legend.position = "none")                             # remove legend

### Visualization functions

We can also create a function to facilitate visualizing different columns in a similar way.

In [ ]:
# Save it to a dataframe
frame_3_mean_sd <- frame_3 %>%
    group_by(k3.cluster) %>%
    select(c(avg_earnings, bottom_25_pctile, top_25_pctile)) %>%
    summarise_all(funs(mean, sd))

# Visualize average earnings by cluster
viz <- function(mean, sd, title) {
    ggplot(frame_3_mean_sd, aes(x=k3.cluster, y=mean, fill=k3.cluster)) +
    geom_bar(position = position_dodge(), stat="identity", fill="gray") +
    geom_errorbar(aes(ymax= mean + sd, ymin = mean),
                  width=.2,
                  position = position_dodge(.9)) +
    ggtitle(title) +
    xlab("Clusters") +
    ylab("Mean") +
    theme(text = element_text(size=16),
          axis.text.x = element_text(size=18, face="bold"),
          legend.position = "none")
}

In [ ]:
viz(frame_3_mean_sd$avg_earnings_mean, frame_3_mean_sd$avg_earnings_sd, "Average Earnings: Differences between clusters")

In [ ]:
viz(frame_3_mean_sd$bottom_25_pctile_mean, frame_3_mean_sd$bottom_25_pctile_sd, "Bottom 25 Percentile: Differences between clusters")

In [ ]:
viz(frame_3_mean_sd$top_25_pctile_mean, frame_3_mean_sd$top_25_pctile_sd, "Top 25 Percentile: Differences between clusters")

### Compare industries

We can also compare clusters by most commonly occuring industries in each cluster.

In [ ]:
# read naics_2017 table into R as dataframe naics
qry = '
select *
from public.naics_2017
'
naics <- dbGetQuery(con, qry)

In [ ]:
frame_3 <- data.frame(emp, k3$cluster)  # add cluster number to the original dataframe

frame_3 <- frame_3 %>%
    group_by(k3.cluster, naics_3_digit) %>%          # group by cluster and industry
    summarise(unique_emp = n_distinct(uiacct)) %>%   # count number of unique employers
    top_n(5, unique_emp)                             # choose top 5 industries in each cluster

# left join with industry names and
frame_3 %>% 
    left_join(naics, by=c('naics_3_digit' = 'naics_us_code')) %>%
    select(-c(seq_no,naics_3_digit)) %>%
    arrange(desc(unique_emp))

What are the most prominent industries in each of the clusters?

Do these clustering results make sense to you? 

## Selecting *k*

How do we know if we chose an optimal number of clusters to describe our data?

### Elbow method

We can use the *Elbow method* to select the optimal cluster number. Recall that *k*-means starts with k random cluster centers (centroids), assigns each data point to the closest centroid, and calculates the distances between each point and the centroid. Then it moves the positions of the centroids and repeats the previous steps until there is convergence. In the *Elbow method*, we try different k values and calculate the sum of squared errors (`SSE`) after the model converges. Then we plot all the `SSE` by K in a line-chart. The line-chart should resemble an arm.

In [ ]:
set.seed(1)

# function to compute total within-cluster sum of square
wss <- function(k) {
    kmeans(emp_ml, k)$tot.withinss
}

# compute and plot wss for k =1 to k = 15
k.values <- 1:15

# extract wss values for each k
wss_values <- map_dbl(k.values, wss)

# plot the resulting SSE for each value of k
plot(k.values, wss_values, 
    type = "b", pch=19, frame=FALSE,
    xlab = "Number of clusters K", 
    ylab = "Total within-clusters sum of squares")

We can see that SSE decreases as we increase k. Here, it decreases faster when k is small. As k increases, the reduction in SSE becomes smaller. We try to choose the number around the inflection point, where the change in SSE becomes negligible, indicating that there is little room to improve the model by increasing k (the bend in the elbow). On our graph, the elbow curve becomes flat around 4.

Let's run the model with 4 clusters.

In [ ]:
set.seed(1)
k4 <- kmeans(emp_ml, centers = 4)
k4$size

We can see that the cluster size with 4 clusters is more evenly distributed now.

Let's save these results to a dataframe called `frame_4`, and check characteristics of employers in each cluster:

In [ ]:
frame_4 <- data.frame(emp, k4$cluster)  # add cluster number to the original dataframe
frame_4 <- subset(frame_4, select= -c(uiacct,naics_3_digit, quarter))  # remove columns with uiacct, naics_3_digit, quarter

frame_4 %>%
    group_by(k4.cluster) %>%
    summarize_all("mean")

We can also take a look at most prominent industries in each cluster.

In [ ]:
# read naics_2017 table into R as naics
qry = '
select *
from public.naics_2017
'
naics <- dbGetQuery(con, qry)

In [ ]:
frame_4 <- data.frame(emp, k4$cluster)  # add cluster number to the original dataframe

frame_4 <- frame_4 %>%
    group_by(k4.cluster, naics_3_digit) %>%
    summarise(unique_emp = n_distinct(uiacct)) %>%
    top_n(5, unique_emp) 

frame_4 %>% 
    left_join(naics, by=c('naics_3_digit' = 'naics_us_code')) %>%
    select(-c(seq_no,naics_3_digit)) 

Which clustering results - `frame_3` or `frame_4` - do you prefer? Do you think it could be optimal to choose more clusters?

In summary, in clustering there is no single right answer - every time we run a different number of clusters, interesting patterns about our data can be exposed. However, what we do want to know is whether the clusters that we find represent true subgroups in our data. This could be a good guidance towards choosing the right number of clusters. (See more information in the Resources section - Professor Steorts, Duke University).

Experiment with different numbers of clusters in the Checkpoint 1 below - given knowledge about Indiana labor market in 2017 Q3, which number of clusters makes most sense to you?

<font color=red><h3> Checkpoint 1: Run a K-Means clustering model </h3></font> 

1. Take a look again at the elbow curve, which number(s) do you think is (are) optimal?

2. Choose a cluster number that you think is best (other than 3 or 4). Use `kmeans()` to run a k-means clustering model with the number you choose. Save your results and features in `frame_k`. 

3. Compare your results with the results we got previously. Do you find any differences? Are the results improved, in your opinion?

Hint: in the Elbow method graph, it looks like 11 could be another optimal cluster - you can try with 11 clusters and see the differences.

### Cohort's Employers

In this section, we will take a look at our cohort's employers, and identify which clusters they belong to based on `frame_4` clustering results.

In [ ]:
# read earnings of 2016 cohort into R
qry = "
select *
from ada_tdc_2020.cohort_2016_earnings
"
df_2016_wages = dbGetQuery(con, qry)

# Subset by 2017 Q3
df_2016_wages <- df_2016_wages[which(df_2016_wages$quarter==3), ]

frame_4 <- data.frame(emp, k4$cluster)  

# Join wages table with frame_4 clustering results
df_2016_wages <- df_2016_wages %>%
    inner_join(frame_4, by='uiacct')

In [ ]:
# Group by clusters and find number of unique employers in each cluster
df_2016_wages %>%
    group_by(k4.cluster) %>%
    summarise(emp_cohort = n_distinct(uiacct))

We can also compare what percentage of all employers in our clusters hire TANF recipients.

In [ ]:
# Get number of unique employers per cluster in the full dataframe (all employers)
frame_4 %>%
    group_by(k4.cluster) %>%
    summarise(emp_all = n_distinct(uiacct))

In [ ]:
# Save cohort and all employers dataframes

cohort_emp <- df_2016_wages %>%
    group_by(k4.cluster) %>%
    summarise(emp_cohort = n_distinct(uiacct))

emp_all <- frame_4 %>%
    group_by(k4.cluster) %>%
    summarise(emp_all = n_distinct(uiacct))

# Join cohort employers with all employers, and find percentage
cohort_emp %>%
    inner_join(emp_all, by = 'k4.cluster') %>%
    mutate(percentage = (emp_cohort / emp_all) * 100)

Let's add industry names:

In [ ]:
# read naics_2017 table into R as naics
qry = '
select *
from public.naics_2017
'
naics <- dbGetQuery(con, qry)

In [ ]:
df_2016_wages_industry_names <- df_2016_wages %>%
    group_by(k4.cluster, naics_3_digit.x) %>%
    summarise(unique_emp = n_distinct(uiacct)) %>%
    slice_max(unique_emp, n = 5) %>%
    arrange(k4.cluster, naics_3_digit.x) %>%
    slice(1:5)         # for cases where there are ties, we need to use slice, to pick only top 5

In [ ]:
df_2016_wages_industry_names %>% 
    left_join(naics, by=c('naics_3_digit.x' = 'naics_us_code')) %>%
    select(-c(seq_no,naics_3_digit.x)) 

We can also compare average earnings of TANF recipients by cluster with average earnings of all employees in each cluster:

In [ ]:
# Calculate average earnings for TANF recipients by cluster
df_2016_wages %>%
    group_by(k4.cluster) %>%
    summarise(mean_earnings_cohort = mean(wages))

In [ ]:
# Calculate average earnings for all employees by cluster
frame_4 %>%
    group_by(k4.cluster) %>%
    summarise(mean_earnings_all = mean(avg_earnings))

<font color=red><h3> Checkpoint 2: Cohort's Employers </h3></font> 

1. How cohort's employers are distributed between clusters in other clustering models (numbers of clusters) that you tried in Checkpoint 1?

2. You can reproduce the clustering process for 2009 cohort, using `employers_2009` table (for Indiana's labor market in 2009 - remember to pick just one quarter), and for 2009 cohort (`cohort_2009_earnings` table).

### Resources:
- UC Business Analytics R Programming Guide: https://uc-r.github.io/kmeans_clustering
- Rebecca Steorts, Assistant Professor, Duke University, Department of Statistical Science, Data Mining and Machine Learning course: https://github.com/resteorts/data-mine/tree/master/lectures_2018/10-unsupervise/10-kmeans.pdf